In [ ]:
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils

using CairoMakie
using LinearAlgebra

In [ ]:
# Operators 
const n_levels = 2
at = create(n_levels)
a = annihilate(n_levels)

H_operators = Dict(
        "X" => a + at,
        "Y" => -im * (a - at),
        "Z" => I - 2 * at * a,
)

# Time
T = 50
Δt = 0.2
;

In [ ]:
# H_drift = zeros(n_levels^2, n_levels^2)
# H_controls = [
#     kron_from_dict("XI", H_operators),
#     kron_from_dict("IX", H_operators),
#     kron_from_dict("YI", H_operators),
#     kron_from_dict("IY", H_operators),
# ]

# H_drift = zeros(n_levels^3, n_levels^3)
# H_controls = [
#     kron_from_dict("XII", H_operators),
#     kron_from_dict("IXI", H_operators),
#     kron_from_dict("IIX", H_operators),
#     kron_from_dict("YII", H_operators),
#     kron_from_dict("IYI", H_operators),
#     kron_from_dict("IIY", H_operators),
# ]

H_drift = zeros(n_levels^4, n_levels^4)
H_controls = [
    kron_from_dict("XIII", H_operators),
    kron_from_dict("IXII", H_operators),
    kron_from_dict("IIXI", H_operators),
    kron_from_dict("IIIX", H_operators),
    # kron_from_dict("YIII", H_operators),
    # kron_from_dict("IYII", H_operators),
    # kron_from_dict("IIYI", H_operators),
    # kron_from_dict("IIIY", H_operators)
]

X_gate = GATES[:X]
SX_gate = sqrt(GATES[:X])
# U_goal = reduce(kron, [X_gate, X_gate])
# U_goal = reduce(kron, [X_gate, X_gate, SX_gate])
U_goal = reduce(kron, [X_gate, X_gate, SX_gate, SX_gate])
;

In [ ]:
prob = UnitarySmoothPulseProblem(
    H_drift,
    H_controls,
    U_goal,
    T,
    Δt;
    Δt_min=Δt / 1.5,
    Δt_max=1.5 * Δt,
    hessian_approximation=true,
    pade_order=4,
)

In [ ]:
solve!(prob; max_iter=25)

unitary_fidelity(prob)

In [ ]:
objective = DefaultObjective()
objective = QuadraticRegularizer(:dda, prob.trajectory, 1e-2)
# objective += QuadraticRegularizer(:da, prob.trajectory, 1.0)
objective += QuadraticRegularizer(:a, prob.trajectory, 1e-2)

# Add acceleration constraint during loop
traj = copy(prob.trajectory)
update_bound!(traj, :a, 1.5)
# update_bound!(traj, :dda, 0.1)
constraints = trajectory_constraints(traj)
# constraints = AbstractConstraint[]

ipopt_options = Options()
ipopt_options.hessian_approximation = "limited-memory"

min_prob = UnitaryMinimumTimeProblem(
    prob;
    objective=objective,
    constraints=constraints,
    final_fidelity=0.99, 
    verbose=false,
    build_trajectory_constraints=false,
    hessian_approximation=true,
    ipopt_options=ipopt_options
)

In [ ]:
solve!(min_prob; max_iter=50)

unitary_fidelity(min_prob)

In [ ]:
# H_crosstalk = (
#     kron_from_dict("ZZ", H_operators)
# )

# H_crosstalk = (
#     kron_from_dict("ZZI", H_operators)
#     + kron_from_dict("IZZ", H_operators)
# )

H_crosstalk = (
    kron_from_dict("ZZII", H_operators)
    + kron_from_dict("IZZI", H_operators)
    + kron_from_dict("IIZZ", H_operators)
    + kron_from_dict("ZIIZ", H_operators)
    + kron_from_dict("ZIII", H_operators)
    + kron_from_dict("IZII", H_operators)
    + kron_from_dict("IIZI", H_operators)
    + kron_from_dict("IIIZ", H_operators)
)

objective = DefaultObjective()
objective = QuadraticRegularizer(:dda, prob.trajectory, 1e-6)
objective += QuadraticRegularizer(:a, prob.trajectory, 1e-6)

# Add acceleration constraint during loop
# traj = copy(prob.trajectory)
# update_bound!(traj, :a, 1.5)
# update_bound!(traj, :dda, 0.1)
# constraints = trajectory_constraints(traj)
constraints = AbstractConstraint[]

ipopt_options = Options()
ipopt_options.hessian_approximation = "limited-memory"

rob_prob_1 = UnitaryRobustnessProblem(
    H_crosstalk,
    prob;
    objective=objective,
    constraints=constraints,
    final_fidelity=0.99, 
    verbose=false,
    build_trajectory_constraints=true,
    hessian_approximation=true,
    ipopt_options=ipopt_options
)

In [ ]:
solve!(rob_prob_1; max_iter=50)

unitary_fidelity(rob_prob_1)

In [ ]:
objective = DefaultObjective()
objective = QuadraticRegularizer(:dda, min_prob.trajectory, 1e-6)
objective += QuadraticRegularizer(:a, min_prob.trajectory, 1e-6)

constraints = AbstractConstraint[]

ipopt_options = Options()
ipopt_options.hessian_approximation = "limited-memory"

rob_prob_2 = UnitaryRobustnessProblem(
    H_crosstalk,
    min_prob;
    objective=objective,
    constraints=constraints,
    final_fidelity=0.99, 
    verbose=false,
    build_trajectory_constraints=true,
    hessian_approximation=true,
    ipopt_options=ipopt_options
)

In [ ]:
solve!(rob_prob_2; max_iter=50)

unitary_fidelity(rob_prob_2)

In [ ]:
@views function infidelity_robustness(Hₑ::AbstractMatrix, p::QuantumControlProblem)
    Z⃗ = vec(p.trajectory.data)
    Z = p.trajectory
    return InfidelityRobustnessObjective(Hₑ, Z).L(Z⃗, Z)
end

In [ ]:
infidelity_robustness(H_crosstalk, min_prob)

In [ ]:
infidelity_robustness(H_crosstalk, rob_prob_1)

In [ ]:
infidelity_robustness(H_crosstalk, rob_prob_2)

In [ ]:
# plot(prob.trajectory; comps=[:a, :da, :dda])
f = Figure(resolution=(800, 600))
ax1 = Axis(f[1, 1])
ax2 = Axis(f[2, 1])
ax3 = Axis(f[3, 1])
ts = accumulate(+, timesteps(min_prob.trajectory))
as = min_prob.trajectory[:a]
das = min_prob.trajectory[:da]
ddas = min_prob.trajectory[:dda]
colors = [:red, :green, :blue, :orange, :purple, :yellow, :black]

for i in 1:size(as, 1)
    lines!(ax1, ts, as[i, :]; color=colors[i])
    lines!(ax2, ts, das[i, :]; color=colors[i])
    lines!(ax3, ts, ddas[i, :]; color=colors[i])
end
f


In [ ]:
# plot(prob.trajectory; comps=[:a, :da, :dda])
f = Figure(resolution=(800, 600))
ax1 = Axis(f[1, 1])
ax2 = Axis(f[2, 1])
ax3 = Axis(f[3, 1])
ts = accumulate(+, timesteps(min_prob.trajectory))
as = min_prob.trajectory[:a]
das = min_prob.trajectory[:da]
ddas = min_prob.trajectory[:dda]
colors = [:red, :green, :blue, :orange, :purple, :yellow, :black]

for i in 1:size(as, 1)
    lines!(ax1, ts, as[i, :]; color=colors[i])
    lines!(ax2, ts, das[i, :]; color=colors[i])
    lines!(ax3, ts, ddas[i, :]; color=colors[i])
end
f

In [ ]:
# plot(prob.trajectory; comps=[:a, :da, :dda])
f = Figure(resolution=(800, 600))
ax1 = Axis(f[1, 1])
ax2 = Axis(f[2, 1])
ax3 = Axis(f[3, 1])
ts = accumulate(+, timesteps(rob_prob_1.trajectory))
as = rob_prob_1.trajectory[:a]
das = rob_prob_1.trajectory[:da]
ddas = rob_prob_1.trajectory[:dda]
colors = [:red, :green, :blue, :orange, :purple, :yellow, :black]

for i in 1:size(as, 1)
    lines!(ax1, ts, as[i, :]; color=colors[i])
    lines!(ax2, ts, das[i, :]; color=colors[i])
    lines!(ax3, ts, ddas[i, :]; color=colors[i])
end
f


In [ ]:
# plot(prob.trajectory; comps=[:a, :da, :dda])
f = Figure(resolution=(800, 600))
ax1 = Axis(f[1, 1])
ax2 = Axis(f[2, 1])
ax3 = Axis(f[3, 1])
ts = accumulate(+, timesteps(rob_prob_2.trajectory))
as = rob_prob_2.trajectory[:a]
das = rob_prob_2.trajectory[:da]
ddas = rob_prob_2.trajectory[:dda]
colors = [:red, :green, :blue, :orange, :purple, :yellow, :black]

for i in 1:size(as, 1)
    lines!(ax1, ts, as[i, :]; color=colors[i])
    lines!(ax2, ts, das[i, :]; color=colors[i])
    lines!(ax3, ts, ddas[i, :]; color=colors[i])
end
f
